<a href="https://colab.research.google.com/github/EricRoh-kr/st9_youtubevid/blob/main/st9_youtubevid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# 필요한 라이브러리 설치 및 로드
import subprocess, sys, os, zipfile, shutil, glob
from datetime import datetime
from IPython.display import display, FileLink, clear_output
import ipywidgets as widgets

# Colab 감지
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

IS_RUNNING = False


urls_input = widgets.Textarea(
    placeholder="유튜브 링크를 한 줄씩 입력하세요",
    layout=widgets.Layout(width="90%", height="120px")
)
quality_choice = widgets.RadioButtons(
    options=["기본 다운로드", "최고 화질 다운로드"],
    description="화질 모드:", style={'description_width': 'initial'}
)
quality_description = widgets.HTML("""
<div style="border:1px solid #ccc; padding:10px; font-size:14px; line-height:1.5;">
<b>기본 다운로드 : </b> 1080p 이하에서 가능한 가장 좋은 화질로 다운로드하며, 속도가 비교적 빠릅니다.<br>
<b>최고 화질 다운로드 : </b> 제공 가능한 최고 화질로 다운로드한 뒤, 필요한 변환 과정을 거칩니다. 속도가 느리고 용량이 커질 수 있습니다.<br><br>
하단의 <b>"▶ 영상 다운로드"</b> 버튼을 누르게 되면 요청 링크들의 영상을 다운, 취합하여 압축 파일로 다운받게 됩니다.
</div>
""")
button = widgets.Button(
    description="▶ 영상 다운로드", button_style="success",
    layout=widgets.Layout(width="200px", height="40px")
)
out = widgets.Output(layout=widgets.Layout(width="90%", border="1px solid #ddd", padding="10px"))

def install_ytdlp_ffmpeg():
    try:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-U", "yt-dlp"],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )
    except subprocess.CalledProcessError:
        return False

    if IN_COLAB:
        subprocess.run(["apt-get", "install", "-y", "ffmpeg"], stdout=subprocess.DEVNULL)

    # 이제 설치가 끝났으니 YoutubeDL 을 import
    global YoutubeDL
    from yt_dlp import YoutubeDL

    return True

def make_progress_hook(out, idx, total, url):
    def hook(d):
        status = d.get('status')
        if status == 'downloading':
            percent = d.get('_percent_str', '').strip()
            speed   = d.get('_speed_str', '')
            eta     = d.get('_eta_str', '')
            # 다운로드 중일 때만 화면 리셋
            out.clear_output(wait=True)
            with out:
                print(f"⬇️ [{idx}/{total}] 다운로드 중: {url}")
                print(f"   진행률: {percent} | 속도: {speed} | 남은 시간: {eta}")
        elif status == 'finished':
            # 완료는 화면을 지우지 않고 메시지 추가
            with out:
                print(f"✅ [{idx}/{total}] 다운로드 완료: {url}\n")
    return hook


def download_videos(urls, folder, mode, out):
    total = len(urls)
    for i, u in enumerate(urls, 1):
        clean_url = u.split('&')[0]
        hook = make_progress_hook(out, i, total, clean_url)
        probe_opts = {'quiet': True, 'no_warnings': True, 'skip_download': True}
        with YoutubeDL(probe_opts) as ydl:
            info = ydl.extract_info(clean_url, download=False)
        w, h = info.get('width', 0), info.get('height', 0)

        if mode == "기본 다운로드":
            if w >= h:
                fmt = "bestvideo[height<=1080]+bestaudio/best"
            else:
                fmt = "bestvideo[width<=1080]+bestaudio/best"
        else:
            fmt = "bestvideo+bestaudio/best"

        ydl_opts = {
            'format': fmt,
            'merge_output_format': 'mp4',
            'outtmpl': f'{folder}/%(title)s.%(ext)s',
            'noplaylist': True,
            'progress_hooks': [hook],
            'quiet': True,
            'no_warnings': True
        }

        with out:
            print(f"\n▶️ [{i}/{total}] 다운로드 시작: {clean_url}")

        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([clean_url])



def make_zip_and_cleanup(folder, out):
    for p in glob.glob("*_유튜브다운로드.zip"):
        try: os.remove(p)
        except: pass

    now = datetime.now().strftime("%m%d_%H%M")
    zip_name = f"{now}_유튜브다운로드.zip"
    with out: print("💾 파일 압축 중...")
    with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(folder):
            for f in files:
                full = os.path.join(root, f)
                zf.write(full, os.path.relpath(full, folder))
    return zip_name

def on_click(b):
    global IS_RUNNING
    if IS_RUNNING: return
    IS_RUNNING = True
    button.disabled = True
    out.clear_output()

    with out:
        urls = [u.strip() for u in urls_input.value.splitlines() if u.strip()]
        if not urls:
            print("❌ 링크를 입력해주세요."); IS_RUNNING=False; button.disabled=False; return

        print("🚀 yt-dlp & ffmpeg 설치 확인...")
        if not install_ytdlp_ffmpeg():
            print("❌ 설치 실패"); IS_RUNNING=False; button.disabled=False; return

        folder = "영상저장함"
        if os.path.exists(folder): shutil.rmtree(folder)
        os.makedirs(folder)

        download_videos(urls, folder, quality_choice.value, out)
        zip_name = make_zip_and_cleanup(folder, out)
        print(f"✅ ZIP 완성: {zip_name}")

        if IN_COLAB: files.download(zip_name)
        else:
            print("👉 링크 클릭 후 다운로드:")
            display(FileLink(zip_name))

    IS_RUNNING = False
    button.disabled = False

button.on_click(on_click)

clear_output(wait=True)
display(widgets.VBox([
    widgets.HTML("<h2>🎬 전략 9팀 YouTube 다운로더</h2>"),
    urls_input, quality_choice, quality_description,
    button, widgets.HTML("<hr><h3>진행 상황</h3>"), out
]))